In [1]:
from dolfin import *
import random

In [2]:
# Input parameter
# Material
D = 100                      # diffusion coefficient
a = Constant((0.0, 0.0))     # advection velocity
lambda_g = 2000              # growth parameter of tumor
lambda_a = 500              # apoptosis parameter of tumor
chi = 1

# Phase field
epsilon = 0.02               # interface width
M = 1.0                      # mobility parameter

# Domain
R = 0.025                    # initial radius of tumor
pts = [0, 0, 1, 1]           # corner points of computational domain
n = 128                      # number of elements in each direction
sig_D = 1.0                  # dirichlet boundary condition of nutrients
sig_0 = 1.0                  # initial condition of nutrients

# Time
dt = 5.0e-06                 # time step size
theta = 0.5                  # time stepping family
nT = 251                     # number of time steps
T = dt*nT

In [3]:
# Define Initial condition as a subclass of UserExpression
class InitialCondition(UserExpression):
    def eval(self, value, x):
        if ((x[0]-(pts[0]+pts[2])/2)**2 + (x[1]-(pts[1]+pts[3])/2)**2) - R >= DOLFIN_EPS:
            value[0] = 0.0     # value phi
            value[1] = 0.0     # value mu
        else: 
            value[0] = 1.0
            value[1] = 0.0
    def value_shape(self):
        return(2,)

In [4]:
# Create mesh and build function space
mesh = RectangleMesh(Point(pts[0], pts[1]), Point(pts[2], pts[3]), n, n)
P1  = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1])
ME = FunctionSpace(mesh, element)

In [5]:
# Define trial and test functions
du = TrialFunction(ME)
q, v = TestFunctions(ME)

In [6]:
# Define solution (current and previous) as function of ME
u = Function(ME)      # current solution
u0 = Function(ME)     # solution from previous converged time step

# Split mixed functions
dphi, dmu = split(du)
phi, mu = split(u)
phi0, mu0 = split(u0)

In [7]:
# Create instance of initial conditions and interpolate 
u_init = InitialCondition(degree=1)
u_init_interp = interpolate(u_init, ME)
u.vector()[:] = u_init_interp.vector()[:]
u0.vector()[:] = u_init_interp.vector()[:]

In [8]:
# Compute the chemical potential df/dphi
phi = variable(phi)     # Define phi as a variable
f = 100*phi**2*(1-phi)**2
dfdphi = diff(f, phi)   # Compute functional derivitive of f

In [9]:
# Time stepping variable
mu_mid = (1.0-theta)*mu0 + theta*mu
dx = Measure('dx', mesh)

# State weak formulation of the governing equations 
L0 = (phi-phi0)*q*dx + M*dt*dot(grad(mu_mid), grad(q))*dx - dt*lambda_g*phi*q*dx + dt*lambda_a*phi*q*dx
L1 = mu*v*dx - dfdphi*v*dx - epsilon*dot(grad(phi), grad(v))*dx
L = L0 + L1

In [10]:
# Compute directional derivative about u in the direction of du (Jacobian)
a = derivative(L, u, du)

In [11]:
# Create output file and specify save location
file1 = File("CH_Growth/phi.pvd", "compressed")
u.rename("phasefield", "u")

In [12]:
# Compute transient solution and write to file in every time step
t = 0      # Starting time
while (t < T):
    t += dt
    u0.vector()[:] = u.vector()
    solve(L==0, u, J=a)

    file1 << (u.split()[0], t)